In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

In [30]:
# define parameters 

tickers = [ 'AAPL',  'MSFT',  'GOOG',  'AMZN',  'TSLA',  'UNH',  'JNJ',  'V',  'XOM',  'TSM',  'NVDA',  'WMT', 
           'JPM',  'PG',  'LLY',  'MA',  'HD',  'CVX',  'META',  'PFE',  'NVO',  'NONOF',  'ABBV',  'MRK',  'KO',
            'BAC', 'PEP',  'BABAF',  'BABA',  'AVGO',  'TMO', 'MCD',  'CSCO',  'DHR',  'SHEL',  'NVS',  'TM',  'ABT',
          ]

ticker = tickers[1]
startDate = '2022-01-01'
endDate = '2022-12-31'
dataInterval = '1d'


# get data
df = yf.download(ticker, startDate, endDate, interval = dataInterval)
df.reset_index(inplace=True)

df['Date'] = df['Date'].dt.date

df['close_ROI'] = df['Close'] / df['Close'].shift(1) - 1
df['Volumne_avg_20'] = df['Volume'].rolling(20).mean()
df['close_ROI_nextday'] = df['close_ROI'].shift(1)

# does todays volume exceed 90th pct
df['Volumne_90th_pct'] = 'No'
df.loc[df['Volume'] >= df['Volumne_avg_20'].quantile(.90),'Volumne_90th_pct'] = 'Yes'

# does todays ROI exceed 90th pct
df['close_ROI_90th_pct'] = 'No'
df.loc[abs(df['close_ROI']) >= abs(df['close_ROI']).quantile(.90),'close_ROI_90th_pct'] = 'Yes'

[*********************100%***********************]  1 of 1 completed


In [31]:
df.groupby(['Volumne_90th_pct','close_ROI_90th_pct']).describe()['close_ROI_nextday']

count      mean       std       min  \
Volumne_90th_pct close_ROI_90th_pct                                        
No               No                  179.0  0.001475  0.022163 -0.054978   
                 Yes                  11.0 -0.003877  0.016450 -0.024249   
Yes              No                   32.0 -0.009861  0.024210 -0.077156   
                 Yes                  13.0 -0.011417  0.022281 -0.044564   

                                          25%       50%       75%       max  
Volumne_90th_pct close_ROI_90th_pct                                          
No               No                 -0.011096 -0.000364  0.015276  0.082268  
                 Yes                -0.013781 -0.010971  0.009688  0.022633  
Yes              No                 -0.027007 -0.007662  0.004320  0.028493  
                 Yes                -0.025893 -0.019050  0.010445  0.029101

In [29]:
data = df[(df['Volumne_90th_pct']=='Yes')&(df['close_ROI_90th_pct']=='Yes')]
data[data['close_ROI']<= -0.05]

,Date,Open,High,Low,Close,Adj Close,Volume,close_ROI,Volumne_avg_20,Volumne_90th_pct,close_ROI_90th_pct
205,2022-10-26,231.169998,238.300003,230.059998,231.320007,230.669937,82543200,-0.077156,31248725.0,Yes,Yes
